In [1]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

Based on an exploratory analysis, it was decided to use the data from the beginning of Jan 2021 to the end of Feb 2024 to exclude the impact of Covid-19 and infantry years of the bike-share program which lacks any clear seasonality. In this period the fluctuations are most likely associated with factors beyond our domain knowledge. To prepare the data for the predictive analysis, several steps were taken.

## Loading the historical bay area bike share data and weather data

The bike share data, gbfs.csv, was downloaded using the historical data scraper from January 2021 to February 2024 and theit wasather data was downloaded for a nearby weather station from the NOAA website,including the same period from January 2021 to February 2024.

**NOTE: the gbfs.csv is not available in this repository due to its large size (>1GB) and needs to be downloaded using the scraper. 

In [2]:
rides_data = pd.read_csv(os.path.join('raw_data','gbfs.csv'))
weather_data = pd.read_csv(os.path.join('raw_data','weather.csv'))

In [3]:
# rides_data['start_lng'].isna().sum()
# rides_data['start_lat'].isna().sum()
# rides_data['end_lng'].isna().sum()
# rides_data['end_lat'].isna().sum()

## Handelling the missing data

The gbfs data contains about 7 million rows, from which 1.2 million are missing start and end station id's. 
I began by addressing missing station IDs (start_station_id and end_station_id) in the gbfs dataset. Recognizing that not all rides had associated station IDs, I used the geographical coordinates (latitude and longitude) of the rides' start and end points to match each ride with the nearest station. 

The main idea is to create a dictionary of known stations and assign the points with the missing station id to the nearest known station id to augment the data. 

In [4]:
rides_data.dropna(subset=['start_lat', 'start_lng', 'end_lat', 'end_lng'], how='any', inplace=True)
rides_data = rides_data.reset_index(drop=True)

gdf_rides_start = gpd.GeoDataFrame(rides_data, geometry=gpd.points_from_xy(rides_data.start_lng, rides_data.start_lat))
gdf_rides_end = gpd.GeoDataFrame(rides_data, geometry=gpd.points_from_xy(rides_data.end_lng, rides_data.end_lat))

known_stations = pd.concat([
    rides_data[['start_station_id', 'start_lat', 'start_lng']].rename(columns={'start_station_id': 'station_id', 'start_lat': 'lat', 'start_lng': 'lng'}),
    rides_data[['end_station_id', 'end_lat', 'end_lng']].rename(columns={'end_station_id': 'station_id', 'end_lat': 'lat', 'end_lng': 'lng'})
]).drop_duplicates().dropna(subset=['station_id', 'lat', 'lng'])

known_stations = known_stations.groupby('station_id').agg({'lat':'mean','lng':'mean'}).reset_index()

gdf_stations = gpd.GeoDataFrame(known_stations, geometry=gpd.points_from_xy(known_stations.lng, known_stations.lat),crs="EPSG:4326")

In [5]:
missing_start = rides_data[rides_data['start_station_id'].isna()]
missing_end = rides_data[rides_data['end_station_id'].isna()]

gdf_missing_start = gpd.GeoDataFrame(missing_start, geometry=gpd.points_from_xy(missing_start.start_lng, missing_start.start_lat), crs="EPSG:4326")
gdf_missing_end = gpd.GeoDataFrame(missing_end, geometry=gpd.points_from_xy(missing_end.end_lng, missing_end.end_lat), crs="EPSG:4326")

nearest_start_stations = gpd.sjoin_nearest(gdf_missing_start, gdf_stations, how='left', distance_col='distance')
nearest_end_stations = gpd.sjoin_nearest(gdf_missing_end, gdf_stations, how='left', distance_col='distance')

/Users/erfanhajibandeh/anaconda3/envs/gbfs/lib/python3.12/site-packages/geopandas/array.py:365: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
/Users/erfanhajibandeh/anaconda3/envs/gbfs/lib/python3.12/site-packages/geopandas/array.py:365: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [6]:
# print(missing_start.shape[0])
# print(nearest_start_stations.shape[0])
# print(missing_end.shape[0])
# print(nearest_end_stations.shape[0])

In [7]:
nearest_start_stations = nearest_start_stations.drop_duplicates(keep='first')
nearest_end_stations = nearest_end_stations.drop_duplicates(keep='first')

In [8]:
for idx, row in nearest_start_stations.iterrows():
    rides_data.at[idx, 'start_station_id'] = row['station_id']

for idx, row in nearest_end_stations.iterrows():
    rides_data.at[idx, 'end_station_id'] = row['station_id']

## Clustering 

### Clustering data to sub regions

Visual inspection of the data revealed three distinct regions within the ride data, suggesting a natural geographical division. To formally identify these regions and reduce the impact of noisy or inappropriate coordinates, I applied a DBSCAN clustering algorithm, which excels in finding high-density areas and marking low-density points as outliers. This step was crucial for eliminating noise and outliers that could skew the analysis or predictive modeling. Careful steps were taken to only exclude the data that were true outliers. 

In [9]:
coords = gdf_stations[['lat', 'lng']].values
scaler = StandardScaler()
coords_scaled = scaler.fit_transform(coords)
dbscan = DBSCAN(eps=0.008, min_samples=2)  # Adjust these parameters as needed
gdf_stations['region'] = dbscan.fit_predict(coords_scaled)

gdf_stations_outliers = gdf_stations[gdf_stations['region'] == -1]
gdf_stations = gdf_stations[gdf_stations['region'] != -1]

In [10]:
# gdf_stations_outliers

In [11]:
station_id_to_cluster = gdf_stations.set_index('station_id')['region'].to_dict()
rides_data['start_region'] = rides_data['start_station_id'].map(station_id_to_cluster)
rides_data['end_region'] = rides_data['end_station_id'].map(station_id_to_cluster)

In [12]:
rides_data = rides_data[(rides_data['end_region'] != -1) | (rides_data['start_region'] != -1)]
rides_data.drop(columns=['start_region','end_region'])
rides_data = rides_data.reset_index(drop=True)

### Clustering region's data to sub clusters

The stations within the three identified regions were further clustered into 30 sub-clusters, distributed proportionally based on the number of stations in each region. The total number of sub-clusters, 30, was chosen based on the geospatial distribution of stations among the three regions. This division was accomplished using KMeans clustering, chosen for its effectiveness in partitioning data into a specified number of clusters. The proportion of sub-clusters assigned to each region was determined by the relative frequency of stations, ensuring that areas with more stations—and presumably more ride activity—were allocated more sub-clusters to accurately reflect their complexity.

In [13]:
station_counts_per_region = gdf_stations['region'].value_counts().sort_index()
total_stations = station_counts_per_region.sum()
proportions = station_counts_per_region / total_stations
sub_cluster_counts = np.round(proportions * 30).astype(int)

In [14]:
gdf_stations.reset_index(drop=True, inplace=True)

sub_clusters = pd.DataFrame(index=gdf_stations.index, columns=['sub_region_cluster'])

for region in station_counts_per_region.index:

    region_stations = gdf_stations[gdf_stations['region'] == region]
    
    n_clusters = sub_cluster_counts[region]
    
    if n_clusters > 0:  
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        sub_cluster_labels = kmeans.fit_predict(region_stations[['lat', 'lng']])
        
        sub_clusters.loc[region_stations.index, 'sub_region_cluster'] = sub_cluster_labels

gdf_stations['sub_region_cluster'] = sub_clusters['sub_region_cluster']

In [15]:
gdf_stations['region'] = gdf_stations['region'].astype(int)
gdf_stations['sub_region_cluster'] = gdf_stations['sub_region_cluster'].astype(int)

In [16]:
gdf_stations['cluster_id'] = gdf_stations['region'].astype(str) + '_' + gdf_stations['sub_region_cluster'].astype(str)
station_id_to_cluster_id = gdf_stations.set_index('station_id')['cluster_id'].to_dict()

In [17]:
rides_data['start_cluster_id'] = rides_data['start_station_id'].map(station_id_to_cluster_id)
rides_data['end_cluster_id'] = rides_data['end_station_id'].map(station_id_to_cluster_id)

In [18]:
rides_data['started_at'] = pd.to_datetime(rides_data['started_at'])
rides_data['date'] = rides_data['started_at'].dt.date
demand = rides_data.groupby(['date', 'start_cluster_id']).size().reset_index(name='demand')
supply = rides_data.groupby(['date', 'end_cluster_id']).size().reset_index(name='supply')

## Incorporation of the weather data

Recognizing the potential impact of weather conditions on ride patterns, I joined the gbfs data with daily weather data to include the average daily temperature for each day in the analysis. This integration enables the exploration of how weather variability influences ride demand and supply (more importantly change in the season and impact on seasonality), providing a more comprehensive dataset for predictive modeling.

In [19]:
weather_data['date'] = pd.to_datetime(weather_data['DATE']).dt.date
avg_temp = weather_data.groupby('date')['TAVG'].mean().reset_index()

## Definition of Supply and Demand

For the purpose of the prediction analysis, I defined "demand" as the total number of rides originating from each cluster and "supply" as the number of rides ending in each cluster. This approach allow us to quantify and compare ride flows within and between the identified clusters, offering insights into patterns of bike usage across different areas.

In [20]:
demand = demand.merge(avg_temp, on='date', how='left')
supply = supply.merge(avg_temp, on='date', how='left')
final_df = demand.merge(supply, left_on=['date', 'start_cluster_id'], right_on=['date', 'end_cluster_id'], how='outer', suffixes=('_demand', '_supply'))
final_df['demand'] = final_df['demand'].fillna(0)
final_df['supply'] = final_df['supply'].fillna(0)
final_df.rename(columns={'start_cluster_id': 'cluster_id', 'TAVG_demand': 'temperature'}, inplace=True)
final_df.drop(['end_cluster_id', 'TAVG_supply'], axis=1, inplace=True)

## Output

Three .csv files are exported from this kernel,including:

stations.csv that includes the station coordiantes, their name and the related cluster. 

model.csv that contains the daily supply, demand, and average temperature data for each cluster and will be used for forecas modelling.  

rides_data which is essentially the cleaned version of the gbfs data as explained in this kernel with the addition of start and end cluster id's. 

In [27]:
final_df = final_df.dropna(subset=['cluster_id'])
final_df.to_csv('model.csv',index=False)

In [ ]:
gdf_stations.drop(columns=['geometry']).to_csv('stations.csv',index=False)

In [ ]:
rides_data[['start_station_id','end_station_id','start_region','end_region','start_cluster_id','end_cluster_id','date']].to_csv('rides.csv',index=False)